# GR4J example

In [2]:
from pathlib import Path
import pandas as pd
import datetime
from hydrogr import InputDataHandler, ModelGr4j
import plotly.graph_objects as go

## Prepare the input data

Here we load a dataframe that contains daily rainfall and evapotranspiration data. The data is already daily sampled.
For demonstration purpose, we will not run the model on all the dataset period, but on a sub-period from 1989 to 1999.

In [3]:
data_path = Path.cwd().parent / 'data'
df = pd.read_pickle(data_path / 'L0123001.pkl')
df.columns = ['date', 'precipitation', 'temperature', 'evapotranspiration', 'flow', 'flow_mm']
df.index = df['date']

inputs = InputDataHandler(ModelGr4j, df)
start_date = datetime.datetime(1989, 1, 1, 0, 0)
end_date = datetime.datetime(1999, 12, 31, 0, 0)
inputs = inputs.get_sub_period(start_date, end_date)

## Setup the model

Here we define the model parameters and the initial states.
Model parameters are mandatory at model creation, and can be changed afterward using the set_parameters method.

In [6]:
    # Set the model :
    parameters = [257.238, 1.012, 88.235, 2.208]
    model = ModelGr4j(parameters)
    model.set_parameters(parameters)  # Re-define the parameters for demonstration purpose.
    
    # Initial state :
    initial_states = {
        "production_store": 0.5,
        "routing_store": 0.6,
        "uh1": None,
        "uh2": None
    }
    model.set_states(initial_states)

## Run the model and display the results

In [7]:
outputs = model.run(inputs.data)
print(outputs.head())

                flow
date                
1989-01-01  1.861714
1989-01-02  1.986144
1989-01-03  1.917419
1989-01-04  1.643920
1989-01-05  1.440313


In [8]:
fig = go.Figure([
    go.Scatter(x=outputs.index, y=outputs['flow'], name="Calculated"),
    go.Scatter(x=inputs.data.index, y=inputs.data['flow_mm'], name="Observed"),
])
fig.show()